NEON file downloaded: NEON_D17_SOAP_DP3_298000_4107000_reflectance
This is just for 2019, I believe the data was gathered in June
There is no provisional data

Data citation:

NEON (National Ecological Observatory Network). Spectrometer orthorectified surface directional reflectance - mosaic (DP3.30006.001), RELEASE-2025. https://doi.org/10.48443/49kq-8q12. Dataset accessed from https://data.neonscience.org/data-products/DP3.30006.001/RELEASE-2025 on April 28, 2025.

This notebook follows this [NEON Data Skills Python Tutorial](https://github.com/NEONScience/NEON-Data-Skills/blob/main/tutorials/Python/AOP/Hyperspectral/intro-hyperspectral/intro-refl-h5/intro-l3-refl-h5.md).

In [ ]:
#import required packages
import os
import requests
import numpy as np
import h5py
from osgeo import gdal
import matplotlib.pyplot as plt

In [ ]:
# display the contents in the ./data folder to confirm the download completed
os.listdir('./data')

I got this error when I ran the cell below: NotADirectoryError: [WinError 267] The directory name is invalid: './data/NEON_refl-surf-dir-ortho-mosaic.zip'.
I assume this is because it is a zip file.
To get around this for now, I moved the hdf5 file to the data folder

In [ ]:
# display the contents in the ./data/NEON_refl-surf-dir-ortho-mosaic.zip folder to confirm the download completed
#os.listdir('./data/NEON_refl-surf-dir-ortho-mosaic.zip')

In [ ]:
# display the contents in the ./data folder to confirm the hdf5 file moved correctly
os.listdir('./data')

## Read in the hdf5 file

In [ ]:
# see what the h5py function does
help(h5py)

In [ ]:
# learn about h5py.File
h5py.File?

In [ ]:
# Note that you may need to update this filepath for your local machine
# the 'r' indicates that this is in readonly mode
f = h5py.File('./data/NEON_D17_SOAP_DP3_298000_4107000_reflectance.h5','r')

# check file
f

## Explore NEON AOP HDF5 Reflectance Files

In [ ]:
#list_dataset lists the names of datasets in an hdf5 file
def list_dataset(name,node):
    if isinstance(node, h5py.Dataset):
        print(name)

f.visititems(list_dataset)

*from the tutorial:*

>You can see that there is a lot of information stored inside this reflectance hdf5 file. Most of this information is metadata (data about the reflectance data), for example, this file stores input parameters used in the atmospheric correction. For this introductory lesson, we will only work with two of these datasets, the reflectance data (hyperspectral cube), and the corresponding geospatial information, stored in Metadata/Coordinate_System:

> * SERC/Reflectance/Reflectance_Data
>   
> * SERC/Reflectance/Metadata/Coordinate_System/

> We can also display the name, shape, and type of each of these datasets using the ls_dataset function defined below, which is also called with the visititems method:

In [ ]:
#ls_dataset displays the name, shape, and type of datasets in hdf5 file
def ls_dataset(name,node):
    if isinstance(node, h5py.Dataset):
        print(node)

In [ ]:
f.visititems(ls_dataset)

## Extract the reflectance data
The reflectance data is nested under SOAP/Reflectance/Reflectance_Data:

In [ ]:
soap_refl = f['SOAP']['Reflectance']
print(soap_refl)

2 members = Metadata and Reflectance_Data.

We will safe reflectance data as soap_reflarray

In [ ]:
soap_reflarray = soap_refl['Reflectance_Data']
print(soap_reflarray)

Use shape method to extract the size of this reflectance array:

In [ ]:
refl_shape = soap_reflarray.shape
print('SOAP Reflectance Data Dimensions:',refl_shape)

This is a 3D shape. The three dimensions are (y,x,bands).

(x,y) are dimensions of the reflectance array in pixels. 

*from the tutorial:*

>"Hyperspectral data sets are often called "cubes" to reflect this 3-dimensional shape."
>
>"NEON hyperspectral data contain around 426 spectral bands, and when working with tiled data, the spatial dimensions are 1000 x 1000, where each pixel represents 1 meter."

## Explore Wavelength Values

Start by extracting wavelength info from soap_refl

In [ ]:
#define the wavelengths variable
wavelengths = soap_refl['Metadata']['Spectral_Data']['Wavelength']

#View wavelength information and values
print('wavelengths:',wavelengths)

Use numpy to see min and max wavelength values:

In [ ]:
# Display min & max wavelengths
print('min wavelength:', np.amin(wavelengths),'nm')
print('max wavelength:', np.amax(wavelengths),'nm')

Find band widths (the distance btw center bands of two adjacent bands). We will do this for the first and last two bands.

*from the tutorial:*

>Remember that Python uses 0-based indexing ([0] represents the first value in an array), and note that you can also use negative numbers to splice values from the end of an array ([-1] represents the last value in an array).

In [ ]:
#show the band widths between the first 2 bands and last 2 bands 
print('band width between first 2 bands =',(wavelengths[1]-wavelengths[0]),'nm')
print('band width between last 2 bands =',(wavelengths[-1]-wavelengths[-2]),'nm')

*from the tutorial:*

>The center wavelengths recorded in this hyperspectral cube range from 383.88 - 2512.18 nm, and each band covers a range of ~5 nm. Now let's extract spatial information, which is stored under SOAP/Reflectance/Metadata/Coordinate_System/Map_Info:

In [ ]:
soap_mapInfo = soap_refl['Metadata']['Coordinate_System']['Map_Info']
print('SOAP Map Info:',soap_mapInfo)

In [ ]:
soap_mapInfo[()]

The cell above yields spatial info abt the reflectance data:

* UTM = CRS
* 1.000, 1.000 - not sure what these are
* 298000.00, 4108000.0 - UTM coordinates (in meters) of the map origin, which refers to the upper-left corner of the image (xMin, yMax)
* 1.0000000, 1.0000000 - pixel resolution in meters
* 11 = UTM zone
* North = UTM Hemisphere (All NEON sites are N)
* WGS-84 = reference ellipoid



*from the tutorial:*

>Note that the letter b that appears before UTM signifies that the variable-length string data is stored in binary format when it is written to the hdf5 file. Don't worry about it for now, as we will convert the numerical data we need into floating point numbers. For more information on hdf5 strings read the h5py documentation.
>
> You can display this in as a string as follows:
> serc_mapInfo[()].decode("utf-8")
>
>Let's extract relevant information from the Map_Info metadata to define the spatial extent of this dataset. To do this, we can use the split method to break up this string into separate values:

In [ ]:
#First convert mapInfo to a string
mapInfo_string = soap_mapInfo[()].decode("utf-8") # read in as string

In [ ]:
#split the strings using the separator "," 
mapInfo_split = mapInfo_string.split(",") 
print(mapInfo_split)

By doing that, we can extract spatial info from map info values and convert them to the float data type and store them so we can access and apply them later when we want to plot the data:

In [ ]:
#Extract the resolution & convert to floating decimal number
res = float(mapInfo_split[5]),float(mapInfo_split[6])
print('Resolution:',res)

In [ ]:
#Extract the upper left-hand corner coordinates from mapInfo
xMin = float(mapInfo_split[3]) 
yMax = float(mapInfo_split[4])

#Calculate the xMax and yMin values from the dimensions
xMax = xMin + (refl_shape[1]*res[0]) #xMax = left edge + (# of columns * x pixel resolution)
yMin = yMax - (refl_shape[0]*res[1]) #yMin = top edge - (# of rows * y pixel resolution)

Next, we will define the spatial extent as a tuple (xMin, xMax, yMin, yMax) as this format is needed to apply the spatial extent when plotting w/ matplotlib.pyplot.

In [ ]:
#Define extent as a tuple:
soap_ext = (xMin, xMax, yMin, yMax)
print('soap_ext:',soap_ext)
print('soap_ext type:',type(soap_ext))

## Extract a Single Band from Array
We will extract a single band that represents a 5nm band (about) that approximates a single wavelength. This band is going to be extracted from the hyperspectral cube using splicing.

The band we extract will be a 2D array that is 1000 x 1000. This array is the scaled reflectance data corresponding to wavelength band 56 (55 in the code as Python indexing starts at 0).

We first have to cast the reflectance data into float values.

In [ ]:
b56 = soap_reflarray[:,:,55].astype(float)
print('b56 type:',type(b56))
print('b56 shape:',b56.shape)
print('Band 56 Reflectance:\n',b56)

## Clean the single band: Scale factor and No Data Value

NEON AOP reflectance data uses a Data_Ignore_Value of -9999 to represent NaN values and a reflectance Scale_Factor of 10000.0 to save disk space.

Extract & apply the Data_Ignore_Value and Scale_Factor:

In [ ]:
#View and apply scale factor and data ignore value
scaleFactor = soap_reflarray.attrs['Scale_Factor']
noDataValue = soap_reflarray.attrs['Data_Ignore_Value']
print('Scale Factor:',scaleFactor)
print('Data Ignore Value:',noDataValue)

b56[b56==int(noDataValue)]=np.nan
b56 = b56/scaleFactor
print('Cleaned Band 56 Reflectance:\n',b56)

## Plot single reflectance band

Use matplotlib.pyplot (plt). Default colormap is jet, different colormaps [here](https://matplotlib.org/2.0.2/examples/color/colormaps_reference.html).

In [ ]:
soap_56_plot = plt.imshow(b56,extent=soap_ext,cmap='Greys') 

Since this image is sort of washed out, we can look at range and distribution of reflectance values that are being plotted w/ a histogram.

## Plot histogram

Use matplotlib.pyplot.hist fxn. This fxn doesn't work if there are NaN values so the code below ignores NaN values.

In [ ]:
plt.hist(b56[~np.isnan(b56)],50); #50 signifies the # of bins

Most of the reflectance values are less than 0.2. To show more contrast, adjust the colorlimit (clim) to 0-0.2:

In [ ]:
soap_56_plot = plt.imshow(b56,extent=soap_ext,cmap='Greys',clim=(0,0.2)) 
plt.title('SOAP Band 56 Reflectance');

## Extract band 53 (index 52)

This band is either red, blue, or green.

In [ ]:
b53 = soap_reflarray[:,:,52].astype(float)
print('b53 type:',type(b53))
print('b53 shape:',b53.shape)
print('Band 53 Reflectance:\n',b53)

In [ ]:
# clean band 53
#View and apply scale factor and data ignore value - commented out as this was done above
# scaleFactor = soap_reflarray.attrs['Scale_Factor']
# noDataValue = soap_reflarray.attrs['Data_Ignore_Value']
# print('Scale Factor:',scaleFactor)
# print('Data Ignore Value:',noDataValue)

b53[b53==int(noDataValue)]=np.nan
b53 = b53/scaleFactor
print('Cleaned Band 53 Reflectance:\n',b53)

In [ ]:
soap_53_plot = plt.imshow(b53,extent=soap_ext,cmap='Greys') 

In [ ]:
# use histogram to check values
plt.hist(b53[~np.isnan(b53)],50); #50 signifies the # of bins

In [ ]:
# adjust color limit on plot
soap_53_plot = plt.imshow(b53,extent=soap_ext,cmap='Greys',clim=(0,0.2)) 
plt.title('SOAP Band 53 Reflectance');

## Extract band 35 (index 34)

Either red, blue, or green

In [ ]:
b35 = soap_reflarray[:,:,34].astype(float)
print('b35 type:',type(b35))
print('b53 shape:',b35.shape)
print('Band 35 Reflectance:\n',b35)

In [ ]:
# clean band 35
#View and apply scale factor and data ignore value - commented out as this was done above
# scaleFactor = soap_reflarray.attrs['Scale_Factor']
# noDataValue = soap_reflarray.attrs['Data_Ignore_Value']
# print('Scale Factor:',scaleFactor)
# print('Data Ignore Value:',noDataValue)

b35[b35==int(noDataValue)]=np.nan
b35 = b35/scaleFactor
print('Cleaned Band 35 Reflectance:\n',b35)

In [ ]:
soap_35_plot = plt.imshow(b35,extent=soap_ext,cmap='Greys') 

In [ ]:
# use histogram to check values
plt.hist(b35[~np.isnan(b35)],50); #50 signifies the # of bins

In [ ]:
# adjust color limit on plot
soap_35_plot = plt.imshow(b35,extent=soap_ext,cmap='Greys',clim=(0,0.2)) 
plt.title('SOAP Band 35 Reflectance');

## Extract band 19 (index 18)

Either red, blue, or green

In [ ]:
b19 = soap_reflarray[:,:,18].astype(float)
print('b19 type:',type(b19))
print('b19 shape:',b19.shape)
print('Band 19 Reflectance:\n',b19)

In [ ]:
# clean band 35
#View and apply scale factor and data ignore value - commented out as this was done above
# scaleFactor = soap_reflarray.attrs['Scale_Factor']
# noDataValue = soap_reflarray.attrs['Data_Ignore_Value']
# print('Scale Factor:',scaleFactor)
# print('Data Ignore Value:',noDataValue)

b19[b35==int(noDataValue)]=np.nan
b19 = b19/scaleFactor
print('Cleaned Band 19 Reflectance:\n',b19)

In [ ]:
soap_19_plot = plt.imshow(b19,extent=soap_ext,cmap='Greys') 

In [ ]:
# use histogram to check values
plt.hist(b19[~np.isnan(b19)],50); #50 signifies the # of bins

In [ ]:
# adjust color limit on plot
soap_19_plot = plt.imshow(b19,extent=soap_ext,cmap='Greys',clim=(0,0.2)) 
plt.title('SOAP Band 19 Reflectance');